![logog](https://raw.githubusercontent.com/Pacific-AI-Corp/langtest/main/docs/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Pacific-AI-Corp/langtest/blob/main/demo/tutorials/llm_notebooks/dataset-notebooks/XSum_dataset.ipynb)

**LangTest** is an open-source python library designed to help developers deliver safe and effective Natural Language Processing (NLP) models. Whether you are using **John Snow Labs, Hugging Face, Spacy** models or **OpenAI, Cohere, AI21, Hugging Face Inference API and Azure-OpenAI** based LLMs, it has got you covered. You can test any Named Entity Recognition (NER), Text Classification, fill-mask, Translation model using the library. We also support testing LLMS for Question-Answering, Summarization and text-generation tasks on benchmark datasets. The library supports 60+ out of the box tests. For a complete list of supported test categories, please refer to the [documentation](http://langtest.org/docs/pages/docs/test_categories).

Metrics are calculated by comparing the model's extractions in the original list of sentences against the extractions carried out in the noisy list of sentences. The original annotated labels are not used at any point, we are simply comparing the model against itself in a 2 settings.

# Getting started with LangTest

In [ ]:
!pip install "langtest[openai,transformers,evaluate]"

# Harness and Its Parameters

The Harness class is a testing class for Natural Language Processing (NLP) models. It evaluates the performance of a NLP model on a given task using test data and generates a report with test results.Harness can be imported from the LangTest library in the following way.

In [3]:
#Import Harness from the LangTest library
from langtest import Harness

It imports the Harness class from within the module, that is designed to provide a blueprint or framework for conducting NLP testing, and that instances of the Harness class can be customized or configured for different testing scenarios or environments.

Here is a list of the different parameters that can be passed to the Harness function:

<br/>


| Parameter  | Description |  
| - | - | 
|**task**     |Task for which the model is to be evaluated (question-answering or summarization)|
| **model**     | Specifies the model(s) to be evaluated. This parameter can be provided as either a dictionary or a list of dictionaries. Each dictionary should contain the following keys: <ul><li>model (mandatory): 	PipelineModel or path to a saved model or pretrained pipeline/model from hub.</li><li>hub (mandatory): Hub (library) to use in back-end for loading model from public models hub or from path</li></ul>|
| **data**      | The data to be used for evaluation. A dictionary providing flexibility and options for data sources. It should include the following keys: <ul><li>data_source (mandatory): The source of the data.</li><li>subset (optional): The subset of the data.</li><li>feature_column (optional): The column containing the features.</li><li>target_column (optional): The column containing the target labels.</li><li>split (optional): The data split to be used.</li><li>source (optional): Set to 'huggingface' when loading Hugging Face dataset.</li></ul> |
| **config**    | Configuration for the tests to be performed, specified in the form of a YAML file. |

<br/>
<br/>

# OpenAI Model Testing For Summarization

In this section, we dive into testing of OpenAI models in summarization task.

LangTest supports robustness tests for LLM testing for now.

In [4]:
import os

os.environ["OPENAI_API_KEY"] = "<YOUR_API_KEY>"

## XSum
[XSum: Extreme Summarization](https://paperswithcode.com/dataset/xsum)

**Dataset Summary**

The Extreme Summarization (XSum) dataset is a dataset for evaluation of abstractive single-document summarization systems. The goal is to create a short, one-sentence new summary answering the question “What is the article about?”. The dataset consists of news articles accompanied with a one-sentence summary

**Data Splits**

- `bias` :	Biased set of the XSum dataset, containing 382 document and summary examples.
- `test` :	Testing set from the XSum dataset, containing 1000 document and summary examples.
- `test-tiny` : Truncated version of XSum dataset which contains 50 document and summary examples.

### Setup and Configure Harness

In [7]:
harness = Harness(
                  task="summarization", 
                  model={"model": "gpt-3.5-turbo-instruct","hub":"openai"}, 
                  data={"data_source" :"XSum",
                        "split":"test-tiny"}
                  )

Test Configuration : 
 {
 "model_parameters": {
  "temperature": 0.2,
  "max_tokens": 64
 },
 "tests": {
  "defaults": {
   "min_pass_rate": 1.0
  },
  "robustness": {
   "add_typo": {
    "min_pass_rate": 0.7
   },
   "lowercase": {
    "min_pass_rate": 0.7
   }
  }
 }
}


## Robustness

For tests we used uppercase, Dyslexia Word Swap. Other available robustness tests for QA task are:
* `add_context`
* `add_contraction`
* `add_punctuation`
* `add_typo`
* `add_ocr_typo`
* `american_to_british`
* `british_to_american`
* `lowercase`
* `strip_punctuation`
* `titlecase`
* `uppercase`
* `number_to_word`
* `add_abbreviation`
* `add_speech_to_text_typo`
* `add_slangs`
* `dyslexia_word_swap`
* `multiple_perturbations`
* `adjective_synonym_swap`
* `adjective_antonym_swap`
* `strip_all_punctuation`

You can also set prompts and other model parameters in config. Possible parameters are:
* `user_promt:` Promt to be given to the model.
* `temperature:` Temperature of the model.
* `max_tokens:` Maximum number of output tokens allowed for model.

In [6]:
harness.configure(
{
 "evaluation":{"threshold": 0.50},
 'tests': {'defaults': {'min_pass_rate': 0.65,
                        },
           'robustness': {'uppercase': {'min_pass_rate': 0.66},
                          'dyslexia_word_swap':{'min_pass_rate': 0.60},
                        }
          }
 }
 )

{'evaluation': {'threshold': 0.5},
 'tests': {'defaults': {'min_pass_rate': 0.65},
  'robustness': {'uppercase': {'min_pass_rate': 0.66},
   'dyslexia_word_swap': {'min_pass_rate': 0.6}}}}

➤ The default metric for summarization is `rouge`. The other available metric is `bertscore` which can be initialised using -> `"evaluation":{"metric":"bertscore", "threshold": 0.5}`

➤The default threshold value is `0.50`. If the eval_score is higher than threshold, then the "pass" will be as true.

➤ You can adjust the level of transformation in the sentence by using the "`prob`" parameter, which controls the proportion of words to be changed during robustness tests.

➤ **NOTE** : "`prob`" defaults to 1.0, which means all words will be transformed.
```
harness.configure(
{
 'tests': {
    'defaults': {'min_pass_rate': 0.65},
      'robustness': {
        'uppercase': {'min_pass_rate': 0.66, 'prob': 0.50},
        'dyslexia_word_swap':{'min_pass_rate': 0.60, 'prob': 0.70},
      }
  }
})

```

Here we have configured the harness to perform robustness tests and defined the minimum pass rate for each test.

In [ ]:
harness.data = harness.data[:5]

### Generating the test cases.

In [ ]:
harness.generate()

Generating testcases...: 100%|██████████| 1/1 [00:00<00:00, 5011.12it/s]


In [ ]:
harness.testcases()

,category,test_type,original,test_case
0,robustness,uppercase,The ex-Reading defender denied fraudulent trad...,THE EX-READING DEFENDER DENIED FRAUDULENT TRAD...
1,robustness,uppercase,Voges was forced to retire hurt on 86 after su...,VOGES WAS FORCED TO RETIRE HURT ON 86 AFTER SU...
2,robustness,uppercase,Seven photographs taken in the Norfolk country...,SEVEN PHOTOGRAPHS TAKEN IN THE NORFOLK COUNTRY...
3,robustness,uppercase,"Chris Poole - known as ""moot"" online - created...","CHRIS POOLE - KNOWN AS ""MOOT"" ONLINE - CREATED..."
4,robustness,uppercase,Four police officers were injured in the incid...,FOUR POLICE OFFICERS WERE INJURED IN THE INCID...
5,robustness,dyslexia_word_swap,The ex-Reading defender denied fraudulent trad...,The ex-Reading defender denied fraudulent trad...
6,robustness,dyslexia_word_swap,Voges was forced to retire hurt on 86 after su...,Voges was forced too retire hurt on 86 after s...
7,robustness,dyslexia_word_swap,Seven photographs taken in the Norfolk country...,Seven photographs taken in the Norfolk country...
8,robustness,dyslexia_word_swap,"Chris Poole - known as ""moot"" online - created...","Chris Poole - known as ""moot"" online - created..."
9,robustness,dyslexia_word_swap,Four police officers were injured in the incid...,Four police officers were injured in the incid...


harness.generate() method automatically generates the test cases (based on the provided configuration)

### Running the tests

In [ ]:
harness.run()

Running testcases... : 100%|██████████| 10/10 [00:35<00:00,  3.50s/it]


Called after harness.generate() and is to used to run all the tests.  Returns a pass/fail flag for each test.

### Generated Results

In [ ]:
harness.generated_results()

,category,test_type,original,test_case,expected_result,actual_result,eval_score,pass
0,robustness,uppercase,The ex-Reading defender denied fraudulent trad...,THE EX-READING DEFENDER DENIED FRAUDULENT TRAD...,"Sam Sodje, 37, and his brothers Efe, 44, Brig...","\nFormer Reading defender Sam Sodje, 37, and h...",0.680412,True
1,robustness,uppercase,Voges was forced to retire hurt on 86 after su...,VOGES WAS FORCED TO RETIRE HURT ON 86 AFTER SU...,"Adam Voges, a 37-year-old Australian crickete...","Adam Voges, a 37-year-old Australian crickete...",0.823529,True
2,robustness,uppercase,Seven photographs taken in the Norfolk country...,SEVEN PHOTOGRAPHS TAKEN IN THE NORFOLK COUNTRY...,The June edition of British Vogue will featur...,Seven photographs taken by photographer Josh ...,0.563107,True
3,robustness,uppercase,"Chris Poole - known as ""moot"" online - created...","CHRIS POOLE - KNOWN AS ""MOOT"" ONLINE - CREATED...","Chris Poole, known as ""moot"" online, created ...","\nChris Poole, known as ""Moot"" online, created...",0.640777,True
4,robustness,uppercase,Four police officers were injured in the incid...,FOUR POLICE OFFICERS WERE INJURED IN THE INCID...,Four police officers were injured in an incid...,Four police officers were injured in an incid...,0.747664,True
5,robustness,dyslexia_word_swap,The ex-Reading defender denied fraudulent trad...,The ex-Reading defender denied fraudulent trad...,"Sam Sodje, 37, and his brothers Efe, 44, Brig...","Sam Sodje, 37, and his brothers Efe, 44, Brig...",0.929293,True
6,robustness,dyslexia_word_swap,Voges was forced to retire hurt on 86 after su...,Voges was forced too retire hurt on 86 after s...,"Adam Voges, a 37-year-old Australian crickete...","Adam Voges, 37, has been forced to retire hur...",0.647619,True
7,robustness,dyslexia_word_swap,Seven photographs taken in the Norfolk country...,Seven photographs taken in the Norfolk country...,The June edition of British Vogue will featur...,The June edition of British Vogue will featur...,0.830189,True
8,robustness,dyslexia_word_swap,"Chris Poole - known as ""moot"" online - created...","Chris Poole - known as ""moot"" online - created...","Chris Poole, known online as ""moot"", created ...","Chris Poole, also known as ""moot"" online, cre...",0.633663,True
9,robustness,dyslexia_word_swap,Four police officers were injured in the incid...,Four police officers were injured in the incid...,Four police officers were injured in an incid...,Four police officers were injured in an incid...,1.000000,True


This method returns the generated results in the form of a pandas dataframe, which provides a convenient and easy-to-use format for working with the test results. You can use this method to quickly identify the test cases that failed and to determine where fixes are needed.

### Final Results

We can call `.report()` which summarizes the results giving information about pass and fail counts and overall test pass/fail flag.

In [ ]:
harness.report()

,category,test_type,fail_count,pass_count,pass_rate,minimum_pass_rate,pass
0,robustness,uppercase,0,5,100%,66%,True
1,robustness,dyslexia_word_swap,0,5,100%,60%,True


## Fairness

Available Fairness tests for QA task are:

* `max_gender_rouge1_score`
* `max_gender_rouge2_score`
* `max_gender_rougeL_score`
* `max_gender_rougeLsum_score`
* `min_gender_rouge1_score`
* `min_gender_rouge2_score`
* `min_gender_rougeL_score`
* `min_gender_rougeLsum_score`

In [ ]:
harness = Harness(
                  task="summarization", 
                  model={"model": "gpt-3.5-turbo-instruct","hub":"openai"}, 
                  data={"data_source" :"XSum",
                        "split":"test-tiny"}
                  )

Test Configuration : 
 {
 "model_parameters": {
  "temperature": 0.2,
  "max_tokens": 64
 },
 "tests": {
  "defaults": {
   "min_pass_rate": 1.0
  },
  "robustness": {
   "add_typo": {
    "min_pass_rate": 0.7
   },
   "lowercase": {
    "min_pass_rate": 0.7
   }
  }
 }
}


In [ ]:
harness.configure(
{
 'tests': {'defaults': {'min_pass_rate': 0.65},
           'fairness': {
               'min_gender_rouge1_score': {'min_score': 0.66},
               'min_gender_rouge2_score':{'min_score': 0.60},
               'min_gender_rougeL_score': {'min_score': 0.66},
               'min_gender_rougeLsum_score': {'min_score': 0.66},
               'max_gender_rouge1_score': {'max_score': 0.66},
               'max_gender_rouge2_score':{'max_score': 0.60},
               'max_gender_rougeL_score': {'max_score': 0.66},
               'max_gender_rougeLsum_score': {'max_score': 0.66},




                        }
          }
 }
 )

{'tests': {'defaults': {'min_pass_rate': 0.65},
  'fairness': {'min_gender_rouge1_score': {'min_score': 0.66},
   'min_gender_rouge2_score': {'min_score': 0.6},
   'min_gender_rougeL_score': {'min_score': 0.66},
   'min_gender_rougeLsum_score': {'min_score': 0.66},
   'max_gender_rouge1_score': {'max_score': 0.66},
   'max_gender_rouge2_score': {'max_score': 0.6},
   'max_gender_rougeL_score': {'max_score': 0.66},
   'max_gender_rougeLsum_score': {'max_score': 0.66}}}}

In [ ]:
harness.data = harness.data[:5]

### Generating the Test Cases

In [ ]:
harness.generate()

Generating testcases...: 100%|██████████| 1/1 [00:00<00:00, 5210.32it/s]


In [ ]:
harness.testcases()

,category,test_type,test_case
0,fairness,min_gender_rouge1_score,male
1,fairness,min_gender_rouge1_score,female
2,fairness,min_gender_rouge1_score,unknown
3,fairness,min_gender_rouge2_score,male
4,fairness,min_gender_rouge2_score,female
5,fairness,min_gender_rouge2_score,unknown
6,fairness,min_gender_rougeL_score,male
7,fairness,min_gender_rougeL_score,female
8,fairness,min_gender_rougeL_score,unknown
9,fairness,min_gender_rougeLsum_score,male


### Running the tests

In [ ]:
harness.run()

Running testcases... :   0%|          | 0/24 [00:00<?, ?it/s]

Running testcases... : 100%|██████████| 24/24 [00:23<00:00,  1.03it/s]


### Generated Results

In [ ]:
harness.generated_results()

,category,test_type,test_case,expected_result,actual_result,pass
0,fairness,min_gender_rouge1_score,male,0.66,0.183087,False
1,fairness,min_gender_rouge1_score,female,0.66,0.200000,False
2,fairness,min_gender_rouge1_score,unknown,0.66,1.000000,True
3,fairness,min_gender_rouge2_score,male,0.60,0.034822,False
4,fairness,min_gender_rouge2_score,female,0.60,0.000000,False
5,fairness,min_gender_rouge2_score,unknown,0.60,1.000000,True
6,fairness,min_gender_rougeL_score,male,0.66,0.105373,False
7,fairness,min_gender_rougeL_score,female,0.66,0.171429,False
8,fairness,min_gender_rougeL_score,unknown,0.66,1.000000,True
9,fairness,min_gender_rougeLsum_score,male,0.66,0.105373,False


### Final Results

In [ ]:
harness.report()

,category,test_type,fail_count,pass_count,pass_rate,minimum_pass_rate,pass
0,fairness,min_gender_rouge1_score,2,1,33%,65%,False
1,fairness,min_gender_rouge2_score,2,1,33%,65%,False
2,fairness,min_gender_rougeL_score,2,1,33%,65%,False
3,fairness,min_gender_rougeLsum_score,2,1,33%,65%,False
4,fairness,max_gender_rouge1_score,1,2,67%,65%,True
5,fairness,max_gender_rouge2_score,1,2,67%,65%,True
6,fairness,max_gender_rougeL_score,1,2,67%,65%,True
7,fairness,max_gender_rougeLsum_score,1,2,67%,65%,True


## Accuracy

Available Accuracy tests for QA task are:

* `min_exact_match_score`
* `min_bleu_score`
* `min_rouge1_score`
* `min_rouge2_score`
* `min_rougeL_score`
* `min_rougeLsum_score`

In [ ]:
harness = Harness(
                  task="summarization", 
                  model={"model": "gpt-3.5-turbo-instruct","hub":"openai"}, 
                  data={"data_source" :"XSum",
                        "split":"test-tiny"}
                  )

Test Configuration : 
 {
 "model_parameters": {
  "temperature": 0.2,
  "max_tokens": 64
 },
 "tests": {
  "defaults": {
   "min_pass_rate": 1.0
  },
  "robustness": {
   "add_typo": {
    "min_pass_rate": 0.7
   },
   "lowercase": {
    "min_pass_rate": 0.7
   }
  }
 }
}


In [ ]:
harness.configure(
{
 'tests': {'defaults': {'min_pass_rate': 0.65},
          'accuracy': {'min_exact_match_score': {'min_score': 0.80},
                        'min_rouge1_score':{'min_score': 0.80},
                        'min_rougeL_score':{'min_score': 0.80},
                        'min_bleu_score':{'min_score': 0.80},
                        'min_rouge2_score':{'min_score': 0.80},
                        'min_rougeLsum_score':{'min_score': 0.80}

                        }
          }
 }
 )

{'tests': {'defaults': {'min_pass_rate': 0.65},
  'accuracy': {'min_exact_match_score': {'min_score': 0.8},
   'min_rouge1_score': {'min_score': 0.8},
   'min_rougeL_score': {'min_score': 0.8},
   'min_bleu_score': {'min_score': 0.8},
   'min_rouge2_score': {'min_score': 0.8},
   'min_rougeLsum_score': {'min_score': 0.8}}}}

### Generating the test cases.

In [ ]:
harness.data = harness.data[:5]

In [ ]:
harness.generate()

Generating testcases...: 100%|██████████| 1/1 [00:00<00:00, 1280.31it/s]


In [ ]:
harness.testcases()

,category,test_type
0,accuracy,min_exact_match_score
1,accuracy,min_rouge1_score
2,accuracy,min_rougeL_score
3,accuracy,min_bleu_score
4,accuracy,min_rouge2_score
5,accuracy,min_rougeLsum_score


### Running the tests

In [ ]:
harness.run()

Running testcases... :   0%|          | 0/6 [00:00<?, ?it/s]

Running testcases... :  50%|█████     | 3/6 [00:08<00:06,  2.15s/it]

Running testcases... : 100%|██████████| 6/6 [00:11<00:00,  1.97s/it]


### Generated Results

In [ ]:
harness.generated_results()

,category,test_type,expected_result,actual_result,pass
0,accuracy,min_exact_match_score,0.8,0.000000,False
1,accuracy,min_rouge1_score,0.8,0.202333,False
2,accuracy,min_rougeL_score,0.8,0.147763,False
3,accuracy,min_bleu_score,0.8,0.000000,False
4,accuracy,min_rouge2_score,0.8,0.056580,False
5,accuracy,min_rougeLsum_score,0.8,0.145599,False


### Final Results

In [ ]:
harness.report()

,category,test_type,fail_count,pass_count,pass_rate,minimum_pass_rate,pass
0,accuracy,min_exact_match_score,1,0,0%,65%,False
1,accuracy,min_rouge1_score,1,0,0%,65%,False
2,accuracy,min_rougeL_score,1,0,0%,65%,False
3,accuracy,min_bleu_score,1,0,0%,65%,False
4,accuracy,min_rouge2_score,1,0,0%,65%,False
5,accuracy,min_rougeLsum_score,1,0,0%,65%,False
